In [7]:
"""
Next Steps:
Try XGBoost
Try t-SNE (https://github.com/danielfrg/tsne)
Build ensamble model
Try TPOT
http://fastml.com/revisiting-numerai/
https://medium.com/jim-fleming/notes-on-the-numerai-ml-competition-14e3d42c19f3
https://github.com/jimfleming/numerai
"""

# Imports
import os
import numpy as np
from scipy.ndimage import imread
import cv2
import sklearn.utils
import pandas as pd
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.layers import Dense, Activation, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras import optimizers
from keras import losses

csv_path = "./numerai_datasets/numerai_training_data.csv"
training_data_df = pd.read_csv(csv_path)
training_data_df

,id,era,data_type,feature1,feature2,feature3,feature4,feature5,feature6,feature7,...,feature42,feature43,feature44,feature45,feature46,feature47,feature48,feature49,feature50,target
0,n2b2e3dd163cb422,era1,train,0.43487,0.44645,0.25802,0.37149,0.62235,0.67451,0.68103,...,0.52962,0.42439,0.51680,0.46297,0.57426,0.57946,0.49646,0.48968,0.54194,1
1,n177021a571c94c8,era1,train,0.50038,0.39216,0.38394,0.51213,0.36660,0.46911,0.68204,...,0.51669,0.48445,0.57587,0.59860,0.67558,0.45577,0.80908,0.50287,0.61629,0
2,n7830fa4c0cd8466,era1,train,0.47416,0.34143,0.39528,0.46337,0.72953,0.45962,0.47869,...,0.41458,0.34804,0.29058,0.51382,0.36389,0.80602,0.39253,0.41821,0.58679,0
3,nc594a184cee941b,era1,train,0.48759,0.55903,0.43987,0.38834,0.44650,0.46389,0.70749,...,0.28776,0.42881,0.55402,0.53695,0.48793,0.62432,0.52898,0.49009,0.49557,0
4,nc5ab8667901946a,era1,train,0.23433,0.55499,0.47849,0.56990,0.64945,0.47152,0.62085,...,0.64405,0.32416,0.33193,0.58065,0.44587,0.47770,0.44020,0.47895,0.57978,0
5,n85e624e4714a4ca,era1,train,0.62443,0.72606,0.74083,0.33016,0.48215,0.43777,0.63622,...,0.40700,0.53875,0.45191,0.51101,0.55764,0.48148,0.77119,0.39730,0.51359,0
6,nbf571e3f5e26428,era1,train,0.49737,0.40568,0.44626,0.37294,0.60940,0.53245,0.74669,...,0.75202,0.65000,0.60511,0.42196,0.63994,0.40502,0.57431,0.44325,0.46746,0
7,nd255f1508f34440,era1,train,0.45185,0.53524,0.32240,0.37249,0.53656,0.50443,0.61508,...,0.44034,0.24539,0.54360,0.72660,0.44836,0.62229,0.37843,0.45673,0.77515,1
8,nbe1320c9294a42b,era1,train,0.56572,0.33367,0.46267,0.29998,0.53056,0.63226,0.56788,...,0.54533,0.33662,0.55950,0.56958,0.48165,0.68348,0.42093,0.38755,0.48252,0
9,ne337363dfd894c8,era1,train,0.48321,0.55944,0.57226,0.48187,0.50234,0.46604,0.32587,...,0.34955,0.53530,0.52522,0.37068,0.52626,0.62940,0.60167,0.36379,0.51803,1


In [2]:
# Initialize simple neural network model
model = Sequential()

# Hidden layer 1: 500 neurons, 'relu' activation
model.add(Dense(units=500, input_dim=50))
model.add(Activation('relu'))

# Hidden layer 2: 250 neurons, 'relu' activation
model.add(Dense(units=500))
model.add(Activation('relu'))

# Hidden layer 3: 250 neurons, 'relu' activation
model.add(Dense(units=500))
model.add(Activation('relu'))

# Hidden layer 4: 250 neurons, 'relu' activation
model.add(Dense(units=250))
model.add(Activation('relu'))

# Hidden layer 5: 250 neurons, 'relu' activation
model.add(Dense(units=250))
model.add(Activation('relu'))

# Output layer: 1 neurons, sigmoid
model.add(Dense(units=1))
model.add(Activation('sigmoid'))

optimizer = optimizers.RMSprop()
loss = "binary_crossentropy"
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

In [3]:
TEST_PERCENT = .2
batch_size = 32
EPOCHS = 5
SELECT_SUBSET_PERCENT = 1.0

select_count = int(len(training_data_df)*SELECT_SUBSET_PERCENT)

X = training_data_df.iloc[:select_count,3:-1]
Y = training_data_df.iloc[:select_count,-1:]

test_size = int(len(X) * TEST_PERCENT)

test_X = np.array(X[:test_size])
test_Y = np.array(Y[:test_size])
train_X = np.array(X[test_size:])
train_Y = np.array(Y[test_size:])

In [5]:
model.fit(train_X, train_Y, batch_size=batch_size, epochs=EPOCHS, validation_split=0.2, verbose=1, shuffle=True)

Train on 251912 samples, validate on 62979 samples
Epoch 1/5
251912/251912 [==============================] - 114s 453us/step - loss: 0.6934 - acc: 0.4978 - val_loss: 0.6932 - val_acc: 0.4994
Epoch 2/5
251912/251912 [==============================] - 133s 527us/step - loss: 0.6932 - acc: 0.4993 - val_loss: 0.6932 - val_acc: 0.4994
Epoch 3/5
251912/251912 [==============================] - 120s 478us/step - loss: 0.6932 - acc: 0.4985 - val_loss: 0.6932 - val_acc: 0.5006
Epoch 4/5
251912/251912 [==============================] - 119s 470us/step - loss: 0.6932 - acc: 0.4996 - val_loss: 0.6932 - val_acc: 0.4994
Epoch 5/5
251912/251912 [==============================] - 118s 468us/step - loss: 0.6932 - acc: 0.5005 - val_loss: 0.6932 - val_acc: 0.4994


In [6]:
loss, acc = model.evaluate(test_X, test_Y, batch_size=batch_size, verbose=1)

print('')
print('Got %.2f%% accuracy' % (acc * 100.))

78722/78722 [==============================] - 8s 107us/step

Got 50.02% accuracy
